# Plain-Text CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [2]:
import pandas as pd  # data mangling and transforming
import numpy as np  # handling vectors and matrices
from sklearn.metrics import mean_squared_error  # MSE error metric

# pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adam

Using TensorFlow backend.


### dataset

In [3]:
train_X = np.load(dir_data+'train_Xu.npy')
train_y = np.load(dir_data+'train_yu.npy')

In [4]:
# # use only the next day
# train_y = train_y[:,:1]

In [5]:
print(train_X.shape)
print(train_y.shape)

(994, 7, 1)
(994, 7)


994 Observations for 7 days for 1 Variable!

In [6]:
test_X = np.load(dir_data+'test_Xu.npy')
test_y = np.load(dir_data+'test_yu.npy')

In [7]:
print(test_X.shape)
print(test_y.shape)

(413, 7, 1)
(413, 7)


## Modeling

### Options

In [8]:
# net hyper parameters
epochs = 20
batch_size = 4
lr = 0.001
# dimensions
n_timesteps = train_X.shape[1] 
n_features = train_X.shape[2]
n_outputs = train_y.shape[1] 

### Tensorflow

In [22]:
def train_tf_model(train_X, train_y,
                   batch_size, epochs, lr,
                   n_timesteps, n_features, n_outputs):
    
    # build model
    model = Sequential()
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu', 
                     input_shape=(n_timesteps,n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer=Adam(lr=lr))
    
    # fit the model
    model.fit(train_X, train_y, 
              epochs=epochs, batch_size=batch_size,
              verbose=0)
    
    # return model
    return model

In [23]:
def pred_tf_model(test_X, test_y, model):
    # create predictions
    preds = model.predict(test_X)
    
    # print RMSE
    rmse = round(np.sqrt(mean_squared_error(test_y, preds)), 2)
    print('RMSE for TF model: %s!'%rmse)
    
    # return predictions and RMSE
    return preds, rmse

### Pytorch

In [24]:
# define class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(n_features, 16, 3, stride=1)
        self.l1 = nn.Linear(16*2, 10)
        self.l2 = nn.Linear(10, n_outputs)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = x.view(-1, 16*2) # flatten
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

In [25]:
def train_pt_model(train_X, train_y,
                   batch_size, epochs, lr,
                   n_timesteps, n_features, n_outputs):
    
    # transform data to tensors (reshape dimensions of X)
    train_X = torch.from_numpy(np.array(train_X.reshape(train_X.shape[0], 
                                                        train_X.shape[2], 
                                                        train_X.shape[1]), dtype='float32'))
    train_y = torch.from_numpy(np.array(train_y, dtype='float32'))
    
    
    # create dataloader
    train_Xy = DataLoader(TensorDataset(train_X, train_y), 
                          batch_size, 
                          shuffle=True)
    
    # build model
    model = Net()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = F.mse_loss
    
    # fit the model
    def fit(df, num_epochs, model, loss_fn, opt):
        for epoch in range(num_epochs):
            for xb,yb in df:
                # Generate predictions
                pred = model(xb)
                loss = loss_fn(pred, yb)
                # Perform gradient descent
                opt.zero_grad()
                loss.backward()
                opt.step()
            
    fit(train_Xy, epochs, model, loss_fn, opt)
    
    # return model
    return model

In [26]:
def pred_pt_model(test_X, test_y, model):
    
    # transform data to tensors (reshape dimensions of X)
    test_X = torch.from_numpy(np.array(test_X.reshape(test_X.shape[0], 
                                                      test_X.shape[2], 
                                                      test_X.shape[1]), dtype='float32'))
    
    # create predictions
    preds = model(test_X).data.numpy()

    # print RMSE
    rmse = round(np.sqrt(mean_squared_error(test_y, preds)), 2)
    print('RMSE for PT model: %s!'%rmse)
    
    # return predictions and RMSE
    return preds, rmse
    

#### Comparison

In [30]:
# tensorflow
model_tf = train_tf_model(train_X, train_y, 
                          batch_size, epochs, lr,
                          n_timesteps, n_features, n_outputs)
y_tf, rmse_tf = pred_tf_model(test_X, test_y, model_tf)

RMSE for TF model: 400.31!


In [31]:
# pytorch
model_pt = train_pt_model(train_X, train_y,
                          batch_size, epochs, lr,
                          n_timesteps, n_features, n_outputs)
y_pt, rmse_pt = pred_pt_model(test_X, test_y, model_pt)

RMSE for PT model: 395.36!
